# Powering Up LangChain: Crafting Cutting-Edge Agents with Custom and Predefined Tools

This notebook demonstrates how to build and integrate custom tools using LangChain to enhance data processing tasks. We will explore two examples where the agent interacts differently with the tools. The first example exclusively uses custom-built tools, while the second combines predefined and custom-built tools. This guide assumes familiarity with the concepts of agents and tools in LangChain.

## Our PDF Document for this Blog Post

Check out the [Open PDF Document](./damage_report.pdf) to familiarize yourself with the data we are processing in this demonstration. Please note that the document is a sample and contains fictional data created for illustrative purposes only. This ensures a clear understanding of the examples and techniques discussed without involving real, sensitive information.


## Overview of Examples

- **Example 1: Custom-Built Tools**
  In this example, the agent employs a suite of custom-built tools specifically designed for tasks such as damage detection, cost calculation, and online price checking. This setup demonstrates how tailored solutions can enhance specific data processing workflows.

- **Example 2: Combination of Predefined and Custom-Built Tools**
  The second example showcases the agent's flexibility by integrating a predefined tool, `DuckDuckGoSearchResults`, with two custom-built tools. This combination highlights how agents can leverage both existing resources and specialized solutions to adapt to varied data processing needs.


## Initial Setup: Importing Libraries and Defining Tools
Begin by importing the necessary libraries. Then, dive into creating specialized tools:
- `damage_detector`: Detects and classifies damage described in text.
- `calculator`: Provides cost estimates for repairs.
- `online_checker`: Checks and compares repair costs with online prices.

In [14]:
import os

from dotenv import load_dotenv, find_dotenv
from langchain.agents import AgentExecutor
from langchain.agents import tool
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import (
    OpenAIToolsAgentOutputParser,
)
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.tools.ddg_search import DuckDuckGoSearchRun
from langchain_community.tools.ddg_search.tool import DuckDuckGoSearchResults
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from pyprojroot import here

In [15]:
@tool
def damage_detector(text: str) -> str:
    """
    Identifies and lists damaged items from a text description,
    including the extent of damage when available.
    """

    template = (
        "text: {text} \n\n Instruction: "
        "Create a list of all damaged objects. Indicate, if applicable, "
        "the extent of the damage for each item."
    )

    prompt = PromptTemplate(
        input_variables=["text"],
        template=template,
    )

    summary_chain = prompt | ChatOpenAI(
        openai_api_key=api_key,
        temperature=0,
        model_name="gpt-4",
    )

    summary = summary_chain.invoke({"text": text})

    return summary.content

In [16]:
@tool
def calculator(text: str) -> str:
    """
    Estimates the total costs for damaged objects to interpret
    and compute values based on textual descriptions.
    """

    template = (
        "text: {text} \n\n Instruction: Calculate the total costs"
        "of the damaged objects"
    )

    prompt = PromptTemplate(
        input_variables=["text"],
        template=template,
    )

    calculator_chain = prompt | ChatOpenAI(
        openai_api_key=api_key,
        temperature=0,
        model_name="gpt-4",
    )

    cost = calculator_chain.invoke({"text": text})

    return cost.content

In [17]:
@tool
def online_checker(text: str) -> str:
    """
    Checks on the web weather the price for an item is supposed to be in a reasonable range
    """

    search_engine = DuckDuckGoSearchRun()

    result = search_engine.run(text)

    return result

## Configuring Environment and Loading Data
Securely load API keys and environmental variables using the `dotenv` package, crucial for maintaining security. The dataset consists of descriptions from `damage_report.pdf`, setting the stage for our tool's operations.



In [18]:
load_dotenv(find_dotenv())
api_key = os.getenv("OPENAI_API_KEY")

In [19]:
data = PyPDFLoader(str(here("./damage_report.pdf"))).load()
print(data)

[Document(page_content='Property Damage Report - Storm Damage\nDear Sir/Madam,\nI am writing to report damage incurred to my property during the storm on February 22, 2024. As a\nholder of a home contents insurance policy, I am seeking prompt attention to this matter.\nA severe storm caused significant damage to my residence in the southern part of Cologne,\nspecifically at Mainzer Straße 45, 50678 Köln. The damage includes broken windows allowing water\nto enter and damage various household items including furniture and electronic devices.\nThe following items were affected: \n- Oak living room table, water damage, estimated current value: EUR 200\n- 55-inch Samsung television, water damage, estimated replacement value: EUR 800\n- Lenovo laptop, water damage, estimated replacement value: EUR 1200\n- Various books and decorative items, estimated current value: EUR 150\nI have taken all necessary measures to prevent further damage and ensure safety. I also have\nphotos of the damaged it

## Integrating Tools with LangChain Agent
Define the custom agent and integrate it with the tools outlined above:
- `damage_detector`: Extracts and details damage from claim texts.
- `calculator`: Calculates estimated repair costs.
- `online_checker`: Verifies cost estimates against current market values.



In [20]:
tools = [damage_detector, calculator, online_checker,]
llm = ChatOpenAI(model_name="gpt-4", openai_api_key=api_key, temperature=0)

In [21]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "I am a dedicated assistant for insurance claims handling. "
            "I help process relevant information and provide cost overviews for claims. "
            "However, I do not have access to current events. "
            "It is essential for the claims handler to verify the plausibility of the "
            "provided cost estimations to ensure they are realistic."
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [27]:
llm_with_tools = llm.bind_tools(tools)

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

## Running the Agent to Process Claims
Execute the agent to process a sample insurance claim. This demonstration uses the `AgentExecutor` class to manage tool interactions and outputs, providing a detailed claim analysis.

In [23]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

response = agent_executor.invoke(
    {"input": data[0].page_content}, return_only_outputs=True
)



> Entering new AgentExecutor chain...

Invoking: `damage_detector` with `{'text': 'Oak living room table, water damage, estimated current value: EUR 200\n55-inch Samsung television, water damage, estimated replacement value: EUR 800\nLenovo laptop, water damage, estimated replacement value: EUR 1200\nVarious books and decorative items, estimated current value: EUR 150'}`


1. Oak living room table - Extent of damage: Water damage
2. 55-inch Samsung television - Extent of damage: Water damage
3. Lenovo laptop - Extent of damage: Water damage
4. Various books and decorative items - Extent of damage: Not specified
Invoking: `calculator` with `{'text': 'Oak living room table, water damage, estimated current value: EUR 200\n55-inch Samsung television, water damage, estimated replacement value: EUR 800\nLenovo laptop, water damage, estimated replacement value: EUR 1200\nVarious books and decorative items, estimated current value: EUR 150'}`


The total cost of the damaged objects is EUR 23

## Evaluating Custom Tool Strategy

The outcomes of using a fully custom-built toolset in the first example are as follows:
- **Damage Identification**: The agent first identified damages such as water damage to various items, including furniture and electronics, detailed in textual descriptions.
- **Cost Calculation**: Following damage identification, the agent calculated precise repair costs, totaling EUR 2350 for the assessed items.
- **Price Verification**: Lastly, the agent compared these costs against current market prices to ensure competitiveness, providing a detailed market analysis for comparable items like the 55-inch television.

These results not only demonstrate how custom tools can significantly automate and enhance processing accuracy and efficiency in niche domains like insurance claim handling, but also showcase the agent's capability to autonomously decide the invocation and sequence of tools. Unlike a fixed procedure, the AgentExecutor dynamically adjusted its approach, choosing which tools to invoke based on the context and requirements of the data being processed. This adaptive sequence of tool invocations highlights the agent's intelligence and flexibility, making it a powerful asset for optimizing efficiency and accuracy in automated data processing systems.

## Setting Up for a Mixed Tool Strategy
Transitioning from the first example, we now explore a mixed strategy where the agent utilizes both predefined and custom-built tools. This setup aims to illustrate the flexibility and adaptability of the agent when faced with different tool capabilities and data requirements.

In [24]:
ddg_search = DuckDuckGoSearchResults()
new_tools = [
    damage_detector,
    calculator,
    ddg_search,
]
llm_with_new_tools = llm.bind_tools(new_tools)

In [25]:
new_agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_new_tools
    | OpenAIToolsAgentOutputParser()
)

In [26]:
new_agent_executor = AgentExecutor(
    agent=new_agent, tools=new_tools, verbose=True
)

new_response = new_agent_executor.invoke(
    {"input": data[0].page_content}, return_only_outputs=True
)



> Entering new AgentExecutor chain...

Invoking: `damage_detector` with `{'text': 'Oak living room table, water damage, estimated current value: EUR 200\n55-inch Samsung television, water damage, estimated replacement value: EUR 800\nLenovo laptop, water damage, estimated replacement value: EUR 1200\nVarious books and decorative items, estimated current value: EUR 150'}`


1. Oak living room table - Extent of damage: Water damage
2. 55-inch Samsung television - Extent of damage: Water damage
3. Lenovo laptop - Extent of damage: Water damage
4. Various books and decorative items - Extent of damage: Not specified
Invoking: `calculator` with `{'text': 'Oak living room table, water damage, estimated current value: EUR 200\n55-inch Samsung television, water damage, estimated replacement value: EUR 800\nLenovo laptop, water damage, estimated replacement value: EUR 1200\nVarious books and decorative items, estimated current value: EUR 150'}`


The total cost of the damaged objects is EUR 23

## Evaluating Mixed Tool Strategy

The second example demonstrates the agent's capability to efficiently handle complex scenarios through the selective use of a mixed toolset:
- **Selective Tool Invocation**: The agent chose not to invoke the Search Tool, based on a detailed assessment of the tool's description in relation to the current data context and the outputs from previously invoked tools. This decision highlights the agent's strategic resource use and its ability to omit unnecessary steps.
- **Adaptive Data Processing**: The agent's approach to tool usage is not fixed but adaptive; it dynamically adjusts the invocation order and selection based on each tool's relevance and utility as determined by the ongoing data analysis.

These operations underline three key aspects of the agent's functionality:
1. The agent does not necessarily invoke all available tools; it selectively employs tools based on their applicability to the task.
2. The order of tool invocation is decided by the agent, optimizing the workflow for efficiency and effectiveness.
3. Whether a tool is invoked depends on the tool's description and the outputs of previously invoked tools, ensuring that each step contributes meaningfully to the overall process.

This measured and pragmatic approach to using a diverse toolset marks a significant advancement in automated data processing technologies. It demonstrates how agents can intelligently manage their resources to meet the specific demands of each scenario.

## Conclusion: Advancing Data Processing with Custom and Predefined Tools

This exploration has highlighted the capabilities of LangChain in enhancing data processing tasks through the strategic use of both custom and predefined tools within agents. The first example illustrated the effectiveness of fully custom tools, tailored for specific tasks such as insurance claims processing, demonstrating how such tools can automate and refine complex data handling comprehensively.

The second example underscored the flexibility of LangChain agents in adaptively integrating a diverse set of tools. This adaptability is evident in the agent's selective tool invocation, its decision-making on the order of invocation based on ongoing assessments, and its reliance on both the tool's detailed descriptions and outputs from preceding tools. Such a strategy not only optimizes efficiency but also meets the dynamic needs of various data sets, showing that a hybrid approach can effectively leverage the strengths of both types of tools.

These case studies emphasize three key aspects of agent functionality:
1. Selective invocation of tools based on their applicability and necessity.
2. Dynamic decision-making on the order of tool use to optimize data processing workflows.
3. Dependence on both the tools' descriptions and the outputs of previously used tools to determine their relevance and utility.

This pragmatic and intelligent approach marks LangChain as a robust framework for building agents capable of handling complex and varied data processing scenarios. Developers and data scientists are encouraged to experiment with their own tool combinations and explore how LangChain can be tailored to meet specific operational needs. By leveraging such a flexible system, you can expand the possibilities of automated data processing, achieving remarkable efficiency and accuracy in your projects.

Thank you for reading this blog post. I hope you found it informative and enjoyable. If you've learned something new and are inspired by the possibilities of LangChain, consider exploring further and experimenting with your own projects. Your feedback and experiences would be greatly valued.